<a href="https://colab.research.google.com/github/thkang727/ML2022_practice/blob/main/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

201724404_ 강태환. mushroom data set을 이용한 Ensemble 모델 을 활용한 성능 비교.

In [ ]:
#Library

import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
# VotingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics

#upload data
data = pd.read_csv('agaricus-lepiota.data', encoding='utf-8')

#data set 확인.
data.head(10)

,poisonous,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


### **버섯**에 대한 data set 을 선택했습니다.
해당 data set은 물리적 특성으로 버섯이 독성이 있는지, 식용이 가능한지를 분류한 data set 입니다.<br>
해당 data set의 변수들은 다음과 같습니다.
------------------
cap-shape(모자 모양): bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s<br>
cap-surface(모자 표면): fibrous=f,grooves=g,scaly=y,smooth=s<br>
cap-color(모자 색상): brown=n,buff=b,cinnamon=c,gray=g,green=r, pink=p,purple=u,red=e,white=w,yellow=y<br>
bruises(타박상)?: bruises=t,no=f<br>
odor(냄새): almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s<br>
gill-attachment(아가미 부착형태): attached=a,descending=d,free=f,notched=n<br>
gill-spacing(아가미 간격): close=c,crowded=w,distant=d<br>
gill-size(아가미 크기): broad=b,narrow=n<br>
gill-color(아가미 색): black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e, white=w,yellow=y<br>
stalk-shape(줄기 모양): enlarging=e,tapering=t<br>
stalk-root(줄기 뿌리): bulbous=b,club=c,cup=u,equal=e, rhizomorphs=z,rooted=r,missing=?<br>
stalk-surface-above-ring(줄기 표면 위 링): fibrous=f,scaly=y,silky=k,smooth=s<br>
stalk-surface-below-ring(줄기 표면 아래 링): fibrous=f,scaly=y,silky=k,smooth=s<br>
stalk-color-above-ring(줄기 위 링 색상): brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y<br>
stalk-color-below-ring(줄기 아래 링 색상): brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y<br>
veil-type(베일 유형): partial=p,universal=u<br>
veil-color(베일 색상): brown=n,orange=o,white=w,yellow=y<br>
ring-number(링 번호): none=n,one=o,two=t<br>
ring-type(링 유형): cobwebby=c,evanescent=e,flaring=f,large=l, none=n,pendant=p,sheathing=s,zone=z<br>
spore-print-color(포자 인쇄 색상): black=k,brown=n,buff=b,chocolate=h,green=r, orange=o,purple=u,white=w,yellow=y<br>
population(개체군): abundant=a,clustered=c,numerous=n, scattered=s,several=v,solitary=y<br>
habitat(서식지): grasses=g,leaves=l,meadows=m,paths=p, urban=u,waste=w,woods=d<br>

In [ ]:
#data upload
from sklearn.preprocessing import LabelEncoder

for i in data:
    LE = LabelEncoder()
    LE.fit(data[i])
    data[i] = LE.transform(data[i])

In [ ]:
from sklearn.model_selection import train_test_split

#data 전처리
x = data.iloc[:,1:]
y = data.iloc[:,0]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=22, stratify=y)

data 전처리를 해줍니다.

In [ ]:
lr_clf = LogisticRegression()
knn_clf = KNeighborsClassifier(n_neighbors = 10)
dtr_clf = DecisionTreeClassifier(max_depth = 5, max_leaf_nodes=4, min_samples_leaf=4)
rfc_clf = RandomForestClassifier(n_estimators=12, max_depth=3,random_state=0)

models = [
  lr_clf,
  knn_clf,
  dtr_clf,
  rfc_clf
]

# hard vote
hard_vote  = VotingClassifier(estimators = [('LR', lr_clf),('KNN', knn_clf), ('DTr', dtr_clf), ('rfc', rfc_clf)], voting='hard')

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=156 )

hard_vote.fit(x_train, y_train)
pred = hard_vote.predict(x_test)
#hard_cm = confusion_matrix(y_test, pred)
hard_score = metrics.accuracy_score(y_test, pred)


# soft vote
soft_vote  = VotingClassifier(estimators = [('LR', lr_clf),('KNN', knn_clf), ('DTr', dtr_clf), ('rfc', rfc_clf)], voting='soft')
soft_vote.fit(x_train, y_train)
pred = soft_vote.predict(x_test)
#soft_cm = confusion_matrix(y_test, pred)
soft_score = metrics.accuracy_score(y_test, pred)

for _ in models :
  _.fit(x_train, y_train)
  pred = _.predict(x_test)
  class_name = _.__class__.__name__
  print('{0}'.format(_))

# confusion Metrics 로 정확성 미리 체크.
#   print('Confusion Metrics :\n{0}'.format(confusion_matrix(y_test, pred)))
  print('ans rate : {0:.4f}'.format(metrics.accuracy_score(y_test, pred)))
  print()

# print("Hard_Confusion Metrics :\n", hard_cm)
print("Hard Voting Ensemble ans rate :{0:.4f}".format(hard_score))
# print("soft_Confusion Metrics \n:", soft_cm)
print("Soft Voting Ensemble ans rate :{0:.4f}".format(soft_score))


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegression()
ans rate : 0.9495

KNeighborsClassifier(n_neighbors=10)
ans rate : 0.9963

DecisionTreeClassifier(max_depth=5, max_leaf_nodes=4, min_samples_leaf=4)
ans rate : 0.9040

RandomForestClassifier(max_depth=3, n_estimators=12, random_state=0)
ans rate : 0.9545

Hard Voting Ensemble ans rate :0.9551
Soft Voting Ensemble ans rate :0.9840


### 구현하려는 Ensemble 모델은 투표 기반 앙상블(voting) 입니다.
여러 단일 모델의 estimator를 결합하여 더 좋은 estimator를 만드는 방법입니다.<br>
서로 다른 알고리즘이 도출해 낸 결과물에 대하여 최종 투표하는 방식을 통해 최종 결과를 선택합니다.<br>
또한 voting은 **Hard vote** 와 **soft vote**로 나뉘는데,<br>
**hard vote**는 결과물에 대한 최종 값을 투표해서 결정하고<br>
**soft vote**는 최종 결과물이 나올 확률 값을 다 더해서 최종 결과물에 대한 각각의 확률을 구한 뒤 최종 값을 도출합니다.<br>
해당 앙상블 모델에서는 **선형회귀, 결정트리, 랜덤포레스트, K-이웃** 의 단일모델을 바탕으로 앙상블 모델을 생성했습니다.<br>
또한 각 모델의 confusion tree 및 F1_score와, Hard/Soft vote 방식에 한 값들을 출력했습니다.

단일 모델과 Ensemble method 성능 비교
단일 모델인
LogisticRegression 의 경우 ans rate : 0.9495,
KNeighborsClassifier 의 경우 ans rate : 0.9723
DecisionTreeClassifier()의 경우 ans rate : 0.9040
RandomForestClassifier()의 경우 ans rate : 0.9545

을 나타냈습니다.

Ensemble method 의 경우 
Hard vote 에서는 ans rate :0.9422
Soft vote 에서는 ans rate :0.9655
가 나왔습니다.

KNN의 경우 앙상블 보다 좋은 정확성을 보였으나, 그 이외의 경우 Ensemble method 에서 좋은 정확성을 보임을 알 수 있었습니다.